In [26]:
# Common
import os
import keras
import numpy as np

# Data
from keras.preprocessing.image import ImageDataGenerator

# Data Visualization
import plotly.express as px
import matplotlib.pyplot as plt

# Model
from keras.models import Sequential, load_model
from keras.layers import GlobalAvgPool2D as GAP, Dense, Dropout

# Callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Pre-Trained Model
from tensorflow.keras.applications import ResNet50, ResNet50V2, InceptionV3, Xception
from keras.layers import Conv2D,MaxPool2D,GlobalAveragePooling2D,AveragePooling2D, MaxPooling2D, Flatten

In [15]:
# Class Names
root_path = '/content/drive/MyDrive/shapes/'
class_names = sorted(os.listdir(root_path))
n_classes = len(class_names)
class_names

['circles', 'rectangles', 'stars', 'triangles']

In [16]:
# Class Distribution
class_dis = [len(os.listdir(root_path + name)) for name in class_names]

# Visualization
fig = px.pie(names=class_names, values=class_dis, title="Class Distribution", hole=0.3)
fig.update_layout({'title':{'x':0.5}})
fig.show()

In [17]:
# Initialize Generator
gen = ImageDataGenerator(rescale=1/255., validation_split=0.1)

# Load Data
train_ds = gen.flow_from_directory(root_path, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=8, subset='training')
valid_ds = gen.flow_from_directory(root_path, class_mode='binary', target_size=(256,256), shuffle=True, batch_size=8, subset='validation')

Found 721 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [18]:
def show_images(GRID=[5,5], model=None, size=(20,20), data=train_ds):
    n_rows = GRID[0]
    n_cols = GRID[1]
    n_images = n_cols * n_rows

    i = 1
    plt.figure(figsize=size)
    for images, labels in data:
        id = np.random.randint(len(images))
        image, label = images[id], class_names[int(labels[id])]

        plt.subplot(n_rows, n_cols, i)
        plt.imshow(image)

        if model is None:
            title = f"Class : {label}"
        else:
            pred = class_names[int(np.argmax(model.predict(image[np.newaxis, ...])))]
            title = f"Org : {label}, Pred : {pred}"

        plt.title(title)
        plt.axis('off')

        i+=1
        if i>=(n_images+1):
            break

    plt.tight_layout()
    plt.show()

In [19]:
show_images()

Output hidden; open in https://colab.research.google.com to view.

In [41]:
BATCH_SIZE = 256
SPE = len(train_ds)//BATCH_SIZE

In [42]:
# Pre-Trained Model
base_model = ResNet50V2(input_shape=(256,256,3), include_top=False)
base_model.trainable = False

# Model Architecture
name = "ResNet50V2-Shape-Detector-v2"
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu', kernel_initializer='he_normal'),
    Dropout(0.4),
    Dense(n_classes, activation='softmax')
], name=name)

# Callbacks
cbs = [EarlyStopping(patience=3, restore_best_weights=True), ModelCheckpoint(name + ".h5", save_best_only=True)]

# Model Compiling
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(train_ds, validation_data=valid_ds, callbacks=cbs, epochs=10)

Epoch 1/10
91/91 [==============================] - 122s 1s/step - loss: 0.5515 - accuracy: 0.8017 - val_loss: 0.4519 - val_accuracy: 0.8875
Epoch 2/10
91/91 [==============================] - 118s 1s/step - loss: 0.1255 - accuracy: 0.9626 - val_loss: 0.3288 - val_accuracy: 0.9000
Epoch 3/10
91/91 [==============================] - 118s 1s/step - loss: 0.0615 - accuracy: 0.9806 - val_loss: 0.4497 - val_accuracy: 0.8500
Epoch 4/10
91/91 [==============================] - 117s 1s/step - loss: 0.0354 - accuracy: 0.9903 - val_loss: 0.4052 - val_accuracy: 0.8625
Epoch 5/10
91/91 [==============================] - 118s 1s/step - loss: 0.0345 - accuracy: 0.9903 - val_loss: 0.4950 - val_accuracy: 0.8500


In [ ]:
# Load Model
model_path = '/content/ResNet50-Shape-Detector-v2.h5'
model = load_model(model_path)
model.summary()

In [ ]:
show_images(model=model, data=valid_ds)